In [3]:
import random
import numpy as np
import enum
import time
from termcolor import colored

In [58]:
class LiftActions(enum.Enum):
    SKIP = 0
    UP1 = 1
    UP2 = 2
    DOWN1 = 3
    DOWN2 = 4
    OPEN1 = 5
    OPEN2 = 6
    CLOSE1 = 7
    CLOSE2 = 8

    START = 9
    FINISH = 10

class LiftsPassState(enum.Enum):
    NONE = 0
    IN1 = 1
    IN2 = 2

# class Lif

In [5]:
# входные символы:


In [75]:
class TwoLifts:

    def __init__(self, floors: int, start_state: tuple, print_movments: bool = False):
        self.floors = floors
        self.print_movments = print_movments
        # Состояние: этаж 1 лифта, этаж второго лифта, (0 - нет пассажиров, 1 - они в 1 лифте, 2 - они во 2 лифте)
        self.state = (*start_state, 0)
        
        self.states_dca = {}

        self.move_count = 0
        self.all_move_count = 0

        self.move_command = [None, None]

        self.queue_symb = [None]
        self.current_symb = None

        self.actions = {
            LiftActions.SKIP: None,
            LiftActions.UP1: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] + 1}, {self.state[1]}]") if self.print_movments else None),
            LiftActions.DOWN1: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] - 1}, {self.state[1]}]") if self.print_movments else None),
            LiftActions.OPEN1: lambda: (print("Лифт 1: открыт") if self.print_movments else None),
            LiftActions.CLOSE1: lambda: (print("Лифт 1: закрыт") if self.print_movments else None),
            
            LiftActions.UP2: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] + 1}]") if self.print_movments else None),
            LiftActions.DOWN2: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] - 1}]") if self.print_movments else None),
            LiftActions.OPEN2: lambda: (print("Лифт 2: открыт") if self.print_movments else None),
            LiftActions.CLOSE2: lambda: (print("Лифт 2: закрыт") if self.print_movments else None),

            LiftActions.START: lambda: (self.next_symb()),
            LiftActions.FINISH: lambda: (print(colored(f"Для вызова ({self.move_command}) было затрачено {self.move_count} операций.", "light_green")), self.plus_all_move_count(), self.next_symb())

        }
    def plus_move_count(self):
        self.move_count += 1
    def plus_all_move_count(self):
        self.all_move_count += self.move_count
        self.move_count = 0

    def next_symb(self):
        self.current_symb = self.queue_symb.pop(0)
        self.move_command.pop(0)
        self.move_command.append(self.current_symb)
        # print("append", self.current_symb)

    def run(self, list_commands):
        self.queue_symb = [item for sublist in list_commands for item in sublist] + [None]
        self.next_symb()
        # print(self.state, self.current_symb)
        while (self.current_symb):
            # time.sleep(0.2)
            # print(self.state, self.current_symb)
            next_item = self.states_dca[self.state][self.current_symb]
            next_item["action"]()
            self.state = next_item["state"]
        print(colored(f"Всего операций {self.all_move_count}", "light_green"))

            

    def generate_dca(self):
        for floor1 in range(1, self.floors + 1):
            for floor2 in range(1, self.floors + 1):
                for pass_state in range(0, 3):
                    self.states_dca[floor1, floor2, pass_state] = {}
                    for next_floor in range(1, self.floors + 1):
                        if abs(floor1 - next_floor) <= abs(floor2 - next_floor) and pass_state != 2:
                            if floor1 < next_floor:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1 + 1, floor2, pass_state),
                                    "action": self.actions[LiftActions.UP1]
                                }
                            elif floor1 > next_floor:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1 - 1, floor2, pass_state),
                                    "action": self.actions[LiftActions.DOWN1]
                                }
                            else:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1, floor2, 0 if pass_state == 1 else 1),
                                    "action": (lambda: (self.actions[LiftActions.OPEN1](), self.actions[LiftActions.CLOSE1](), self.actions[LiftActions.FINISH]())) if pass_state != 0 
                                        else (lambda: (self.actions[LiftActions.OPEN1](), self.actions[LiftActions.CLOSE1](), self.actions[LiftActions.START]())),
                                }
                        else: 
                            if floor2 < next_floor:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1, floor2 + 1, pass_state),
                                    "action": self.actions[LiftActions.UP2]
                                }
                            elif floor2 > next_floor:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1, floor2 - 1, pass_state),
                                    "action": self.actions[LiftActions.DOWN2]
                                }
                            else:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1, floor2, 0 if pass_state == 2 else 2),
                                    "action": (lambda: (self.actions[LiftActions.OPEN1](), self.actions[LiftActions.CLOSE1](), self.actions[LiftActions.FINISH]())) if pass_state != 0 
                                        else (lambda: (self.actions[LiftActions.OPEN1](), self.actions[LiftActions.CLOSE1](), self.actions[LiftActions.START]())),
                                }

In [10]:
floors = list(range(1, random.randint(5, 21)))
print(floors[-1])
actions_count = random.randint(5, 6)
actions = [np.random.choice(floors, 2).tolist() for i in range(actions_count)]
print(actions)


7
[[5, 5], [5, 4], [2, 6], [7, 7], [3, 3]]


In [76]:
l = TwoLifts(floors=floors[-1], start_state=(1, 1), print_movments=True)
l.generate_dca()
l.run(actions)

Лифты: [1, 1]->[2, 1]
Лифты: [2, 1]->[3, 1]
Лифты: [3, 1]->[4, 1]
Лифты: [4, 1]->[5, 1]
Лифт 1: открыт
Лифт 1: закрыт
Лифт 1: открыт
Лифт 1: закрыт
Для вызова ([5, 5]) было затрачено 4 операций.
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [5, 1]->[4, 1]
Лифт 1: открыт
Лифт 1: закрыт
Для вызова ([5, 4]) было затрачено 1 операций.
Лифты: [4, 1]->[4, 2]
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [4, 2]->[4, 3]
Лифты: [4, 3]->[4, 4]
Лифты: [4, 4]->[4, 5]
Лифты: [4, 5]->[4, 6]
Лифт 1: открыт
Лифт 1: закрыт
Для вызова ([2, 6]) было затрачено 5 операций.
Лифты: [4, 6]->[4, 7]
Лифт 1: открыт
Лифт 1: закрыт
Лифт 1: открыт
Лифт 1: закрыт
Для вызова ([7, 7]) было затрачено 1 операций.
Лифты: [4, 7]->[3, 7]
Лифт 1: открыт
Лифт 1: закрыт
Лифт 1: открыт
Лифт 1: закрыт
Для вызова ([3, 3]) было затрачено 1 операций.
Всего операций 12


In [44]:
b = 1
a = (lambda: print(1)) if b == 1 else (lambda: print(2))
a()

1
